In [1]:
#General DS Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
%matplotlib inline

# working with dates
from datetime import datetime


#My imports
#warnings

In [2]:
# #get my data from csv files
# df2015 = pd.read_csv("crime-incident-reports-2015.csv")
# df2016 = pd.read_csv("crime-incident-reports-2016.csv")
# df2017 = pd.read_csv("crime-incident-reports-2017.csv")
# df2018 = pd.read_csv("crime-incident-reports-2018.csv")
# df2019 = pd.read_csv("crime-incident-reports-2019.csv")
# df2020 = pd.read_csv("crime-incident-reports-2020.csv")
# df2021 = pd.read_csv("crime-incident-reports-2021.csv")

In [3]:
# #Looking at the data by year
# df2015.info(), df2016.info(), df2017.info(), df2018.info(), df2019.info(), df2020.info(), df2021.info()

In [4]:
# # Forming a table from DFs
# table = []
# table.append(df2015)
# table.append(df2016)
# table.append(df2017)
# table.append(df2018)
# table.append(df2019)
# table.append(df2020)
# table.append(df2021)

In [5]:
# #Combine all tables, ignore index
# df = pd.concat(table, ignore_index=True)

In [6]:
#Save data as combined csv
#df.to_csv('boston_crime.csv')

In [7]:
#Get CSV from file
df = pd.read_csv('boston_crime.csv', index_col=0)

/var/folders/vw/1h05rsrx3wbc05jyk4c2bl680000gn/T/ipykernel_4253/781030722.py:2: DtypeWarning: Columns (1,3,7,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('boston_crime.csv', index_col=0)


In [8]:
#Looking at it all combined
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 583052 entries, 0 to 583051
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   INCIDENT_NUMBER      583052 non-null  object 
 1   OFFENSE_CODE         583052 non-null  int64  
 2   OFFENSE_CODE_GROUP   353253 non-null  object 
 3   OFFENSE_DESCRIPTION  583052 non-null  object 
 4   DISTRICT             578913 non-null  object 
 5   REPORTING_AREA       583052 non-null  object 
 6   SHOOTING             231254 non-null  object 
 7   OCCURRED_ON_DATE     583052 non-null  object 
 8   YEAR                 583052 non-null  int64  
 9   MONTH                583052 non-null  int64  
 10  DAY_OF_WEEK          583052 non-null  object 
 11  HOUR                 583052 non-null  int64  
 12  UCR_PART             353156 non-null  object 
 13  STREET               571165 non-null  object 
 14  Lat                  560522 non-null  float64
 15  Long             

In [9]:
#Peeking at the data
df.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I192068249,2647,Other,THREATS TO DO BODILY HARM,B2,280,NaN,2015-08-28 10:20:00,2015,8,Friday,10,Part Two,WASHINGTON ST,42.330119,-71.084251,"(42.33011862, -71.08425106)"
1,I192061894,1106,Confidence Games,FRAUD - CREDIT CARD / ATM FRAUD,C11,356,NaN,2015-08-20 00:00:00,2015,8,Thursday,0,Part Two,CHARLES ST,42.300605,-71.061268,"(42.30060543, -71.06126785)"
2,I192038828,1107,Fraud,FRAUD - IMPERSONATION,A1,172,NaN,2015-11-02 12:24:00,2015,11,Monday,12,Part Two,ALBANY ST,42.334288,-71.072395,"(42.33428841, -71.07239518)"
3,I192008877,1107,Fraud,FRAUD - IMPERSONATION,E18,525,NaN,2015-07-31 10:00:00,2015,7,Friday,10,Part Two,WINGATE RD,42.237009,-71.129566,"(42.23700950, -71.12956606)"
4,I182090828,1102,Fraud,FRAUD - FALSE PRETENSE / SCHEME,D4,159,NaN,2015-12-01 12:00:00,2015,12,Tuesday,12,Part Two,UPTON ST,42.342432,-71.072258,"(42.34243222, -71.07225766)"


In [10]:
#Rename columns to remove capital letters
#df['INCIDENT_NUMBER'] = df['INCIDENT_NUMBER'].str.lower()
df.columns = df.columns.str.lower()

In [11]:
df.head()

,incident_number,offense_code,offense_code_group,offense_description,district,reporting_area,shooting,occurred_on_date,year,month,day_of_week,hour,ucr_part,street,lat,long,location
0,I192068249,2647,Other,THREATS TO DO BODILY HARM,B2,280,NaN,2015-08-28 10:20:00,2015,8,Friday,10,Part Two,WASHINGTON ST,42.330119,-71.084251,"(42.33011862, -71.08425106)"
1,I192061894,1106,Confidence Games,FRAUD - CREDIT CARD / ATM FRAUD,C11,356,NaN,2015-08-20 00:00:00,2015,8,Thursday,0,Part Two,CHARLES ST,42.300605,-71.061268,"(42.30060543, -71.06126785)"
2,I192038828,1107,Fraud,FRAUD - IMPERSONATION,A1,172,NaN,2015-11-02 12:24:00,2015,11,Monday,12,Part Two,ALBANY ST,42.334288,-71.072395,"(42.33428841, -71.07239518)"
3,I192008877,1107,Fraud,FRAUD - IMPERSONATION,E18,525,NaN,2015-07-31 10:00:00,2015,7,Friday,10,Part Two,WINGATE RD,42.237009,-71.129566,"(42.23700950, -71.12956606)"
4,I182090828,1102,Fraud,FRAUD - FALSE PRETENSE / SCHEME,D4,159,NaN,2015-12-01 12:00:00,2015,12,Tuesday,12,Part Two,UPTON ST,42.342432,-71.072258,"(42.34243222, -71.07225766)"


In [12]:
df.shooting.value_counts()

0    219825
0      7133
1      2779
Y      1455
1        62
Name: shooting, dtype: int64

In [13]:
#Look at columns with mixed data types
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 583052 entries, 0 to 583051
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   incident_number      583052 non-null  object 
 1   offense_code         583052 non-null  int64  
 2   offense_code_group   353253 non-null  object 
 3   offense_description  583052 non-null  object 
 4   district             578913 non-null  object 
 5   reporting_area       583052 non-null  object 
 6   shooting             231254 non-null  object 
 7   occurred_on_date     583052 non-null  object 
 8   year                 583052 non-null  int64  
 9   month                583052 non-null  int64  
 10  day_of_week          583052 non-null  object 
 11  hour                 583052 non-null  int64  
 12  ucr_part             353156 non-null  object 
 13  street               571165 non-null  object 
 14  lat                  560522 non-null  float64
 15  long             

In [14]:
#Looking at Mixed Dtype colums 1, 3, 7, 13
df.offense_code.dtypes

dtype('int64')

In [15]:
#Looking at 7 Occured on date, see if there are nulls
df.isnull().sum()
#The most nulls are in shooting, which we can input with zero (could be incorrect but theres no data)

incident_number             0
offense_code                0
offense_code_group     229799
offense_description         0
district                 4139
reporting_area              0
shooting               351798
occurred_on_date            0
year                        0
month                       0
day_of_week                 0
hour                        0
ucr_part               229896
street                  11887
lat                     22530
long                    22530
location                    0
dtype: int64

In [16]:
#see nulls in shooting where offense code is homicide
len(df[(df.shooting.isnull()) & (df.offense_code == 111)])

45

In [17]:
#Let's replace white space and empty cells with nan values
# Replace a whitespace sequence or empty with a NaN value and reassign this manipulation to df.

df = df.replace(r'^\s*$', np.nan, regex=True)

In [18]:
#Checking offense code group to see what to do with the nulls
df.offense_code_group.value_counts()

Motor Vehicle Accident Response              41064
Larceny                                      28906
Medical Assistance                           26211
Investigate Person                           20420
Other                                        19860
                                             ...  
HUMAN TRAFFICKING                                8
INVESTIGATE PERSON                               5
Burglary - No Property Taken                     5
HUMAN TRAFFICKING - INVOLUNTARY SERVITUDE        2
Biological Threat                                2
Name: offense_code_group, Length: 67, dtype: int64

In [19]:
df[df.offense_code_group.isna()]
#df[df.offense_code_group.isnull()]
#Lets replace with unknown for Nan

,incident_number,offense_code,offense_code_group,offense_description,district,reporting_area,shooting,occurred_on_date,year,month,day_of_week,hour,ucr_part,street,lat,long,location
353253,102091671,2647,NaN,THREATS TO DO BODILY HARM,B3,417,0,2019-11-12 12:00:00,2019,11,Tuesday,12,NaN,MORA ST,42.282082,-71.073649,"(42.28208197671972, -71.07364874515646)"
353254,102095489,3115,NaN,INVESTIGATE PERSON,E18,520,0,2019-11-25 16:30:00,2019,11,Monday,16,NaN,POYDRAS ST,42.256216,-71.124019,"(42.256215920402155, -71.12401947329023)"
353255,102096818,2905,NaN,VAL - VIOLATION OF AUTO LAW,A1,NaN,0,2019-11-30 21:00:00,2019,11,Saturday,21,NaN,SUDBURY ST & CAMBRIDGE ST\nBOSTON MA 02108\nU...,42.360866,-71.061316,"(42.360866027118476, -71.0613160019785)"
353256,129092894,3201,NaN,PROPERTY - LOST/ MISSING,NaN,503,0,2019-11-16 13:30:00,2019,11,Saturday,13,NaN,AMERICAN LEGION HWY,42.284467,-71.111831,"(42.28446742674232, -71.11183088758158)"
353257,129099920,3301,NaN,VERBAL DISPUTE,B2,330,0,2019-12-12 07:50:00,2019,12,Thursday,7,NaN,COLUMBIA ROAD,0.000000,0.000000,"(0, 0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583047,121005808,1102,NaN,FRAUD - FALSE PRETENSE / SCHEME,B3,289,0,2021-01-19 12:00:00,2021,1,Tuesday,12,NaN,CLARKWOOD ST,42.333679,-71.091878,"(42.33367921810846, -71.09187754618458)"
583048,121000816,3304,NaN,NOISY PARTY/RADIO-NO ARREST,E5,675,0,2021-01-04 00:00:00,2021,1,Monday,0,NaN,BRADFIELD AVENUE,0.000000,0.000000,"(0, 0)"
583049,120221042,3006,NaN,SICK/INJURED/MEDICAL - PERSON,E5,680,0,2021-04-02 10:45:00,2021,4,Friday,10,NaN,WASHINGTON ST,42.283902,-71.130863,"(42.283902436483125, -71.13086262643496)"
583050,10637224,3125,NaN,WARRANT ARREST - BOSTON WARRANT (MUST BE SUPPL...,D4,270,0,2021-01-26 18:54:00,2021,1,Tuesday,18,NaN,NORTHAMPTON ST,42.333951,-71.075389,"(42.333951248393774, -71.07538938822692)"


In [20]:
#Replace NaN values in offense code group with unknown
df['offense_code_group'].fillna("unknown", inplace=True)

In [23]:
df[df.offense_code_group == 'unknown']

,incident_number,offense_code,offense_code_group,offense_description,district,reporting_area,shooting,occurred_on_date,year,month,day_of_week,hour,ucr_part,street,lat,long,location
353253,102091671,2647,unknown,THREATS TO DO BODILY HARM,B3,417,0,2019-11-12 12:00:00,2019,11,Tuesday,12,NaN,MORA ST,42.282082,-71.073649,"(42.28208197671972, -71.07364874515646)"
353254,102095489,3115,unknown,INVESTIGATE PERSON,E18,520,0,2019-11-25 16:30:00,2019,11,Monday,16,NaN,POYDRAS ST,42.256216,-71.124019,"(42.256215920402155, -71.12401947329023)"
353255,102096818,2905,unknown,VAL - VIOLATION OF AUTO LAW,A1,NaN,0,2019-11-30 21:00:00,2019,11,Saturday,21,NaN,SUDBURY ST & CAMBRIDGE ST\nBOSTON MA 02108\nU...,42.360866,-71.061316,"(42.360866027118476, -71.0613160019785)"
353256,129092894,3201,unknown,PROPERTY - LOST/ MISSING,NaN,503,0,2019-11-16 13:30:00,2019,11,Saturday,13,NaN,AMERICAN LEGION HWY,42.284467,-71.111831,"(42.28446742674232, -71.11183088758158)"
353257,129099920,3301,unknown,VERBAL DISPUTE,B2,330,0,2019-12-12 07:50:00,2019,12,Thursday,7,NaN,COLUMBIA ROAD,0.000000,0.000000,"(0, 0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583047,121005808,1102,unknown,FRAUD - FALSE PRETENSE / SCHEME,B3,289,0,2021-01-19 12:00:00,2021,1,Tuesday,12,NaN,CLARKWOOD ST,42.333679,-71.091878,"(42.33367921810846, -71.09187754618458)"
583048,121000816,3304,unknown,NOISY PARTY/RADIO-NO ARREST,E5,675,0,2021-01-04 00:00:00,2021,1,Monday,0,NaN,BRADFIELD AVENUE,0.000000,0.000000,"(0, 0)"
583049,120221042,3006,unknown,SICK/INJURED/MEDICAL - PERSON,E5,680,0,2021-04-02 10:45:00,2021,4,Friday,10,NaN,WASHINGTON ST,42.283902,-71.130863,"(42.283902436483125, -71.13086262643496)"
583050,10637224,3125,unknown,WARRANT ARREST - BOSTON WARRANT (MUST BE SUPPL...,D4,270,0,2021-01-26 18:54:00,2021,1,Tuesday,18,NaN,NORTHAMPTON ST,42.333951,-71.075389,"(42.333951248393774, -71.07538938822692)"


In [25]:
#Let's look at shooting data, deal with the nulls

In [26]:
df.shooting.value_counts()

0    219825
0      7133
1      2779
Y      1455
1        62
Name: shooting, dtype: int64

In [27]:
#Changing Y in shooting to 1(Yes)
df.loc[df["shooting"] == "Y", "shooting"] = 1

In [28]:
df.shooting.value_counts()

0    219825
0      7133
1      4234
1        62
Name: shooting, dtype: int64

In [31]:
#Replace Nan Values in shooting with zeroes
df['shooting'].fillna(0, inplace=True)

In [32]:
#Check for nulls, none are left
df.shooting.isnull().sum()

0

In [33]:
#why are there multiple zeroes and ones in my column? Let me recast the column data type as integer
df['shooting'] = df['shooting'].astype(int)
print(df.shooting.dtypes)

int64


In [34]:
df.shooting.value_counts()

0    578756
1      4296
Name: shooting, dtype: int64

In [35]:
#Looks good to me

In [36]:
#Going back to missing values
df.isnull().sum()

incident_number             0
offense_code                0
offense_code_group          0
offense_description         0
district                 4139
reporting_area          55657
shooting                    0
occurred_on_date            0
year                        0
month                       0
day_of_week                 0
hour                        0
ucr_part               229896
street                  11888
lat                     22530
long                    22530
location                    0
dtype: int64

In [37]:
#UCR Part has alot of null values, lets check it out

In [38]:
df.ucr_part.value_counts()

Part Three    176042
Part Two      107656
Part One       68068
Other           1390
Name: ucr_part, dtype: int64

In [39]:
#Lets fill nulls here with unknown, its not going to be important to my analysis
#Replace NaN values in ucr part with unknown
df['ucr_part'].fillna("unknown", inplace=True)

In [41]:
#No more nulls
df.ucr_part.isnull().sum(), df.ucr_part.value_counts()

(0,
 unknown       229896
 Part Three    176042
 Part Two      107656
 Part One       68068
 Other           1390
 Name: ucr_part, dtype: int64)

In [42]:
#Going back and looking at nulls
df.isnull().sum()

incident_number            0
offense_code               0
offense_code_group         0
offense_description        0
district                4139
reporting_area         55657
shooting                   0
occurred_on_date           0
year                       0
month                      0
day_of_week                0
hour                       0
ucr_part                   0
street                 11888
lat                    22530
long                   22530
location                   0
dtype: int64

In [43]:
#Lets check reporting area
df.reporting_area.value_counts()

355    7663
465    6480
282    5794
167    5335
77     5063
       ... 
699      16
190       8
950       2
218       1
666       1
Name: reporting_area, Length: 880, dtype: int64

In [44]:
#Going to fill NaN with zeroes for reporting area, which represents where a crime is reported from
df['reporting_area'].fillna(0, inplace=True)

In [47]:
#No more nulls
df.reporting_area.isnull().sum()

0

In [48]:
df.isnull().sum()

incident_number            0
offense_code               0
offense_code_group         0
offense_description        0
district                4139
reporting_area             0
shooting                   0
occurred_on_date           0
year                       0
month                      0
day_of_week                0
hour                       0
ucr_part                   0
street                 11888
lat                    22530
long                   22530
location                   0
dtype: int64

In [49]:
len(df)

583052

In [50]:
#Number of unique incidents in this data
df.incident_number.nunique()

540979

#### 92.78 percent of the data involve unique incidents, but lets look at incidents with multiple rows

In [54]:
#It's an object
df.incident_number.dtypes

dtype('O')

In [56]:
#find duplicate rows across all columns
duplicateRows = df[df.duplicated()]
duplicateRows

,incident_number,offense_code,offense_code_group,offense_description,district,reporting_area,shooting,occurred_on_date,year,month,day_of_week,hour,ucr_part,street,lat,long,location
1250,I152107190,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,B3,427,1,2015-12-29 08:47:00,2015,12,Tuesday,8,Part One,WILMORE ST,42.277964,-71.092463,"(42.27796370, -71.09246318)"
1252,I152107190,111,Homicide,"MURDER, NON-NEGLIGIENT MANSLAUGHTER",B3,427,1,2015-12-29 08:47:00,2015,12,Tuesday,8,Part One,WILMORE ST,42.277964,-71.092463,"(42.27796370, -71.09246318)"
1254,I152107190,3125,Warrant Arrests,WARRANT ARREST,B3,427,1,2015-12-29 08:47:00,2015,12,Tuesday,8,Part Three,WILMORE ST,42.277964,-71.092463,"(42.27796370, -71.09246318)"
1256,I152107190,1501,Firearm Violations,"WEAPON - FIREARM - CARRYING / POSSESSING, ETC",B3,427,1,2015-12-29 08:47:00,2015,12,Tuesday,8,Part Two,WILMORE ST,42.277964,-71.092463,"(42.27796370, -71.09246318)"
2438,I152105851,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,E13,302,1,2015-12-23 15:39:00,2015,12,Wednesday,15,Part One,CENTRE ST,42.322838,-71.100967,"(42.32283759, -71.10096723)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353245,I070720870-00,3125,Warrant Arrests,WARRANT ARREST,B2,318,0,2018-12-13 00:00:00,2018,12,Thursday,0,Part Three,BROOKLEDGE ST,42.309563,-71.089902,"(42.30956305, -71.08990197)"
353246,I070720870-00,3125,Warrant Arrests,WARRANT ARREST,B2,318,0,2018-12-13 00:00:00,2018,12,Thursday,0,Part Three,BROOKLEDGE ST,42.309563,-71.089902,"(42.30956305, -71.08990197)"
353248,I070720870-00,802,Simple Assault,ASSAULT & BATTERY,B2,318,0,2018-12-13 00:00:00,2018,12,Thursday,0,Part Two,BROOKLEDGE ST,42.309563,-71.089902,"(42.30956305, -71.08990197)"
353249,I070720870-00,3125,Warrant Arrests,WARRANT ARREST,B2,318,0,2018-12-13 00:00:00,2018,12,Thursday,0,Part Three,BROOKLEDGE ST,42.309563,-71.089902,"(42.30956305, -71.08990197)"


In [61]:
#643 rows with supposedly duplicated data
df.duplicated().sum()

643

In [81]:
#Lets check some of them and see whats going on (11 instances)
df[(df['incident_number'] == 'I070720870-00')]

,incident_number,offense_code,offense_code_group,offense_description,district,reporting_area,shooting,occurred_on_date,year,month,day_of_week,hour,ucr_part,street,lat,long,location
353239,I070720870-00,637,Larceny,LARCENY IN A BUILDING UNDER $50,B2,318,0,2018-12-13 00:00:00,2018,12,Thursday,0,Part One,BROOKLEDGE ST,42.309563,-71.089902,"(42.30956305, -71.08990197)"
353240,I070720870-00,637,Larceny,LARCENY IN A BUILDING UNDER $50,B2,318,0,2018-12-13 00:00:00,2018,12,Thursday,0,Part One,BROOKLEDGE ST,42.309563,-71.089902,"(42.30956305, -71.08990197)"
353241,I070720870-00,637,Larceny,LARCENY IN A BUILDING UNDER $50,B2,318,0,2018-12-13 00:00:00,2018,12,Thursday,0,Part One,BROOKLEDGE ST,42.309563,-71.089902,"(42.30956305, -71.08990197)"
353242,I070720870-00,3125,Warrant Arrests,WARRANT ARREST,B2,318,0,2018-12-13 00:00:00,2018,12,Thursday,0,Part Three,BROOKLEDGE ST,42.309563,-71.089902,"(42.30956305, -71.08990197)"
353243,I070720870-00,3125,Warrant Arrests,WARRANT ARREST,B2,318,0,2018-12-13 00:00:00,2018,12,Thursday,0,Part Three,BROOKLEDGE ST,42.309563,-71.089902,"(42.30956305, -71.08990197)"
353244,I070720870-00,3125,Warrant Arrests,WARRANT ARREST,B2,318,0,2018-12-13 00:00:00,2018,12,Thursday,0,Part Three,BROOKLEDGE ST,42.309563,-71.089902,"(42.30956305, -71.08990197)"
353245,I070720870-00,3125,Warrant Arrests,WARRANT ARREST,B2,318,0,2018-12-13 00:00:00,2018,12,Thursday,0,Part Three,BROOKLEDGE ST,42.309563,-71.089902,"(42.30956305, -71.08990197)"
353246,I070720870-00,3125,Warrant Arrests,WARRANT ARREST,B2,318,0,2018-12-13 00:00:00,2018,12,Thursday,0,Part Three,BROOKLEDGE ST,42.309563,-71.089902,"(42.30956305, -71.08990197)"
353247,I070720870-00,802,Simple Assault,ASSAULT & BATTERY,B2,318,0,2018-12-13 00:00:00,2018,12,Thursday,0,Part Two,BROOKLEDGE ST,42.309563,-71.089902,"(42.30956305, -71.08990197)"
353248,I070720870-00,802,Simple Assault,ASSAULT & BATTERY,B2,318,0,2018-12-13 00:00:00,2018,12,Thursday,0,Part Two,BROOKLEDGE ST,42.309563,-71.089902,"(42.30956305, -71.08990197)"


In [76]:
big_incident

0         False
1         False
2         False
3         False
4         False
          ...  
583047      NaN
583048      NaN
583049      NaN
583050      NaN
583051      NaN
Name: incident_number, Length: 583052, dtype: object

In [74]:
#Lets deal with nulls in incident number
df.incident_number.isna()

0         False
1         False
2         False
3         False
4         False
          ...  
583047    False
583048    False
583049    False
583050    False
583051    False
Name: incident_number, Length: 583052, dtype: bool

#### Looking at these rows, there's no way to know if they are errors or real values. Its likely they are one incident with multiple arrests, so therefore I choose to keep them. There's no way to know if we have a bigger error in the collection of these statistics--i.e. this data was collected BETTER than all others

In [82]:
#Going back to looking at nulls
df.isnull().sum()

incident_number            0
offense_code               0
offense_code_group         0
offense_description        0
district                4139
reporting_area             0
shooting                   0
occurred_on_date           0
year                       0
month                      0
day_of_week                0
hour                       0
ucr_part                   0
street                 11888
lat                    22530
long                   22530
location                   0
dtype: int64

In [83]:
# lets look at street
df.street.value_counts()

WASHINGTON ST                                           28564
BLUE HILL AVE                                           14084
BOYLSTON ST                                             11655
HARRISON AVE                                             8974
DORCHESTER AVE                                           8729
                                                        ...  
HASTING                                                     1
RUTHERFORD AVE & CITY SQ\nCHARLESTOWN  MA 02129\nUNI        1
FATHER FRANCIS J GILDAY ST & E NEWTON ST\nBOSTON  M         1
BOWDOIN ST & NORTON ST\nBOSTON  MA 02122\nUNITED STA        1
BRADFIELD AVENUE                                            1
Name: street, Length: 14300, dtype: int64

In [86]:
df[df.street.isnull()]
#Some streets that are null have gps data, I wonder if I could use that. Others have all location data missing--I
#think we should drop those that have nulls in all location fields

,incident_number,offense_code,offense_code_group,offense_description,district,reporting_area,shooting,occurred_on_date,year,month,day_of_week,hour,ucr_part,street,lat,long,location
166,I162041845,3201,Property Lost,PROPERTY - LOST,D4,0,0,2015-09-01 12:00:00,2015,9,Tuesday,12,Part Three,NaN,NaN,NaN,"(0.00000000, 0.00000000)"
217,I162024760,3018,Medical Assistance,SICK/INJURED/MEDICAL - POLICE,D4,0,0,2015-09-28 14:41:00,2015,9,Monday,14,Part Three,NaN,NaN,NaN,"(0.00000000, 0.00000000)"
380,I162005633,3201,Property Lost,PROPERTY - LOST,D4,0,0,2015-11-15 22:00:00,2015,11,Sunday,22,Part Three,NaN,NaN,NaN,"(0.00000000, 0.00000000)"
441,I162003177,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,D4,0,0,2015-12-28 17:55:00,2015,12,Monday,17,Part One,NaN,NaN,NaN,"(0.00000000, 0.00000000)"
537,I162001010,3831,Motor Vehicle Accident Response,M/V - LEAVING SCENE - PROPERTY DAMAGE,C11,0,0,2015-12-31 18:30:00,2015,12,Thursday,18,Part Three,NaN,NaN,NaN,"(0.00000000, 0.00000000)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561495,212028173,3807,unknown,M/V ACCIDENT - OTHER CITY VEHICLE,B3,0,0,2021-05-01 11:05:00,2021,5,Saturday,11,unknown,NaN,42.291741,-71.077179,"(42.29174103255666, -71.07717899069988)"
561516,212028147,3831,unknown,M/V - LEAVING SCENE - PROPERTY DAMAGE,B2,0,0,2021-05-01 08:24:00,2021,5,Saturday,8,unknown,NaN,42.327987,-71.099354,"(42.32798700396488, -71.09935402988641)"
561529,212028130,3006,unknown,SICK/INJURED/MEDICAL - PERSON,A1,111,0,2021-05-01 03:00:00,2021,5,Saturday,3,unknown,NaN,42.352370,-71.062600,"(42.352370011853225, -71.06259996388145)"
561532,212028127,3802,unknown,M/V ACCIDENT - PROPERTY DAMAGE,B2,320,0,2021-05-01 02:33:00,2021,5,Saturday,2,unknown,NaN,42.308468,-71.083277,"(42.30846803058291, -71.08327694723572)"


In [94]:
#this looks right
df[df.street.isnull() & df.lat.isnull() & df.long.isnull()]

,incident_number,offense_code,offense_code_group,offense_description,district,reporting_area,shooting,occurred_on_date,year,month,day_of_week,hour,ucr_part,street,lat,long,location
166,I162041845,3201,Property Lost,PROPERTY - LOST,D4,0,0,2015-09-01 12:00:00,2015,9,Tuesday,12,Part Three,NaN,NaN,NaN,"(0.00000000, 0.00000000)"
217,I162024760,3018,Medical Assistance,SICK/INJURED/MEDICAL - POLICE,D4,0,0,2015-09-28 14:41:00,2015,9,Monday,14,Part Three,NaN,NaN,NaN,"(0.00000000, 0.00000000)"
380,I162005633,3201,Property Lost,PROPERTY - LOST,D4,0,0,2015-11-15 22:00:00,2015,11,Sunday,22,Part Three,NaN,NaN,NaN,"(0.00000000, 0.00000000)"
441,I162003177,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,D4,0,0,2015-12-28 17:55:00,2015,12,Monday,17,Part One,NaN,NaN,NaN,"(0.00000000, 0.00000000)"
537,I162001010,3831,Motor Vehicle Accident Response,M/V - LEAVING SCENE - PROPERTY DAMAGE,C11,0,0,2015-12-31 18:30:00,2015,12,Thursday,18,Part Three,NaN,NaN,NaN,"(0.00000000, 0.00000000)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352726,I182000372,3410,Towed,TOWED MOTOR VEHICLE,C6,0,0,2018-01-02 13:16:00,2018,1,Tuesday,13,Part Three,NaN,NaN,NaN,"(0.00000000, 0.00000000)"
352966,I182000064,3803,Motor Vehicle Accident Response,M/V ACCIDENT - PERSONAL INJURY,B2,0,0,2018-01-01 03:29:00,2018,1,Monday,3,Part Three,NaN,NaN,NaN,"(0.00000000, 0.00000000)"
352967,I182000064,2101,Operating Under the Influence,OPERATING UNDER THE INFLUENCE ALCOHOL,B2,0,0,2018-01-01 03:29:00,2018,1,Monday,3,Part Two,NaN,NaN,NaN,"(0.00000000, 0.00000000)"
353168,I172002908,802,Simple Assault,ASSAULT SIMPLE - BATTERY,A1,0,0,2018-01-16 16:00:00,2018,1,Tuesday,16,Part Two,NaN,NaN,NaN,"(0.00000000, 0.00000000)"


In [99]:
#Creating a df with the missing street, lat and long values
missing_location = df[df.street.isnull() & df.lat.isnull() & df.long.isnull()]

In [100]:
missing_location
#9246 rows, but since we are focusing on location (perhaps), we can afford to drop these rows with null values.
#May be able to impute location based on street if we have SOME location data (besides district)

,incident_number,offense_code,offense_code_group,offense_description,district,reporting_area,shooting,occurred_on_date,year,month,day_of_week,hour,ucr_part,street,lat,long,location
166,I162041845,3201,Property Lost,PROPERTY - LOST,D4,0,0,2015-09-01 12:00:00,2015,9,Tuesday,12,Part Three,NaN,NaN,NaN,"(0.00000000, 0.00000000)"
217,I162024760,3018,Medical Assistance,SICK/INJURED/MEDICAL - POLICE,D4,0,0,2015-09-28 14:41:00,2015,9,Monday,14,Part Three,NaN,NaN,NaN,"(0.00000000, 0.00000000)"
380,I162005633,3201,Property Lost,PROPERTY - LOST,D4,0,0,2015-11-15 22:00:00,2015,11,Sunday,22,Part Three,NaN,NaN,NaN,"(0.00000000, 0.00000000)"
441,I162003177,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,D4,0,0,2015-12-28 17:55:00,2015,12,Monday,17,Part One,NaN,NaN,NaN,"(0.00000000, 0.00000000)"
537,I162001010,3831,Motor Vehicle Accident Response,M/V - LEAVING SCENE - PROPERTY DAMAGE,C11,0,0,2015-12-31 18:30:00,2015,12,Thursday,18,Part Three,NaN,NaN,NaN,"(0.00000000, 0.00000000)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352726,I182000372,3410,Towed,TOWED MOTOR VEHICLE,C6,0,0,2018-01-02 13:16:00,2018,1,Tuesday,13,Part Three,NaN,NaN,NaN,"(0.00000000, 0.00000000)"
352966,I182000064,3803,Motor Vehicle Accident Response,M/V ACCIDENT - PERSONAL INJURY,B2,0,0,2018-01-01 03:29:00,2018,1,Monday,3,Part Three,NaN,NaN,NaN,"(0.00000000, 0.00000000)"
352967,I182000064,2101,Operating Under the Influence,OPERATING UNDER THE INFLUENCE ALCOHOL,B2,0,0,2018-01-01 03:29:00,2018,1,Monday,3,Part Two,NaN,NaN,NaN,"(0.00000000, 0.00000000)"
353168,I172002908,802,Simple Assault,ASSAULT SIMPLE - BATTERY,A1,0,0,2018-01-16 16:00:00,2018,1,Tuesday,16,Part Two,NaN,NaN,NaN,"(0.00000000, 0.00000000)"


In [101]:
#Let's leave this here in case we need to drop
#df.drop(missing_location, inplace = True)